In [1]:
import os
import cv2
import time
import numpy as np

import time
import os
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler

from torch.utils.data import Dataset,DataLoader,Subset
from PIL import Image,ImageOps,ImageEnhance

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import RandomSampler

import torchvision
import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import save_image
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder

from matplotlib import pyplot as plt
plt.ion()   # interactive mode
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
DIR_TRAIN = "/train/directory/path"
DIR_VALID = "/dev/directory/path"

OUTPUT_DIR = './model/output/directory/path'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [3]:
### Exploring Dataset

classes = os.listdir(DIR_TRAIN)
print("Total Classes: ",len(classes))

#Counting total train and valid

train_count = 0
valid_count = 0
test_count = 0
for _class in classes:
    train_count += len(os.listdir(DIR_TRAIN + _class))
    valid_count += len(os.listdir(DIR_VALID + _class))

print("Total train images: ",train_count)
print("Total valid images: ",valid_count)
    

Total Classes:  40
Total train images:  1600000
Total valid images:  400000


In [4]:
def resizer(x):
    x = Image.fromarray(x)
    x = x.resize((299,299), resample= Image.BICUBIC)
    #x = np.asarray(x)
    return x

In [4]:
train_dataset = torchvision.datasets.ImageFolder(root=DIR_TRAIN, transform= T.Compose([
            T.Resize(299),
            T.CenterCrop(299),
            #T.Lambda(resizer),
            T.ToTensor(),
            #T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

test_dataset = torchvision.datasets.ImageFolder(root=DIR_VALID, transform= T.Compose([
            T.Resize(299),
            T.CenterCrop(299),
            #T.Lambda(resizer),
            T.ToTensor(),
            #T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))

image_datasets ={'train':train_dataset, 'val': test_dataset}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=40,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [5]:
class_names = image_datasets['train'].classes
x, y =iter(dataloaders['val']).next()

In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    path = os.path.join(OUTPUT_DIR, 'inception_v3_new.pt')
    torch.save(model, path)
    return model

In [11]:
import timm
model = timm.create_model('inception_v3', pretrained=True, num_classes=40)

In [13]:
device ='cuda'
model_ft = model.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001)

# Decay LR 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [14]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=4)

Epoch 0/3
----------
train Loss: 0.1537 Acc: 0.9496
val Loss: 0.0214 Acc: 0.9930
Epoch 1/3
----------
train Loss: 0.0147 Acc: 0.9952
val Loss: 0.0173 Acc: 0.9942
Epoch 2/3
----------
train Loss: 0.0083 Acc: 0.9973
val Loss: 0.0042 Acc: 0.9986
Epoch 3/3
----------
train Loss: 0.0059 Acc: 0.9980
val Loss: 0.0040 Acc: 0.9987
Training complete in 768m 48s
Best val Acc: 0.998653
